Resources:
https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TAPAS/Fine_tuning_TapasForQuestionAnswering_on_SQA.ipynb#scrollTo=QP4AHMxFujhV


In [1]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd
import datetime

In [19]:
#tokenizer = TapasTokenizer.from_pretrained('google/tapas-base-finetuned-wtq')
tokenizer = TapasTokenizer.from_pretrained('google/tapas-large-finetuned-wtq')
#model = TapasForQuestionAnswering.from_pretrained('google/tapas-base-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-large-finetuned-wtq')

In [11]:
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
import torch

#model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(1024, 1024)
      (token_type_embeddings_0): Embedding(3, 1024)
      (token_type_embeddings_1): Embedding(256, 1024)
      (token_type_embeddings_2): Embedding(256, 1024)
      (token_type_embeddings_3): Embedding(2, 1024)
      (token_type_embeddings_4): Embedding(256, 1024)
      (token_type_embeddings_5): Embedding(256, 1024)
      (token_type_embeddings_6): Embedding(10, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024

In [5]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
cfs_data=pd.read_json(query_str)
print(cfs_data.shape)
cfs_data.sample(5)

The number of incidents in the set is 7553
(7553, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
2399,220190590,T09,22008931,Vehicle Fire,2022-01-19T00:00:00.000,2022-01-18T00:00:00.000,2022-01-19T07:51:16.000,2022-01-19T07:51:48.000,2022-01-19T07:52:18.000,2022-01-19T07:53:41.000,...,1,TRUCK,2,10,10,Bayview Hunters Point,220190590-T09,"{'type': 'Point', 'coordinates': [-122.3971670...",NaN,NaN
526,220171060,SCRT4,22008133,Medical Incident,2022-01-17T00:00:00.000,2022-01-17T00:00:00.000,2022-01-17T10:47:36.000,2022-01-17T10:48:18.000,2022-01-17T10:56:52.000,2022-01-17T10:56:52.000,...,1,SUPPORT,4,2,6,South of Market,220171060-SCRT4,"{'type': 'Point', 'coordinates': [-122.4109992...",NaN,NaN
3118,220201105,E03,22009474,Medical Incident,2022-01-20T00:00:00.000,2022-01-20T00:00:00.000,2022-01-20T10:41:05.000,2022-01-20T10:42:12.000,2022-01-20T10:43:54.000,2022-01-20T10:44:47.000,...,1,ENGINE,1,2,6,Tenderloin,220201105-E03,"{'type': 'Point', 'coordinates': [-122.4125718...",NaN,NaN
3969,220211331,E06,22009974,Medical Incident,2022-01-21T00:00:00.000,2022-01-21T00:00:00.000,2022-01-21T11:49:27.000,2022-01-21T11:49:27.000,2022-01-21T11:50:55.000,2022-01-21T11:52:02.000,...,1,ENGINE,2,2,6,Mission,220211331-E06,"{'type': 'Point', 'coordinates': [-122.4215223...",NaN,NaN
3182,220200645,AM108,22009423,Medical Incident,2022-01-20T00:00:00.000,2022-01-20T00:00:00.000,2022-01-20T08:11:58.000,2022-01-20T08:12:31.000,2022-01-20T08:12:47.000,2022-01-20T08:13:34.000,...,1,PRIVATE,1,3,6,South of Market,220200645-AM108,"{'type': 'Point', 'coordinates': [-122.4020613...",2022-01-20T08:25:55.000,2022-01-20T08:58:13.000


In [14]:
cfs_data=cfs_data.dropna()
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)

cfs_data_tapas=cfs_small[['unit_type','call_type','neighborhoods_analysis_boundaries']]

In [17]:
cfs_data_tapas=cfs_data_tapas.rename(columns={"unit_type":"unit","call_type":"incident","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_data_tapas

,unit,incident,neighbourhood
8,PRIVATE,Medical Incident,Financial District/South Beach
9,MEDIC,Medical Incident,Lone Mountain/USF
22,MEDIC,Medical Incident,Outer Richmond
24,MEDIC,Medical Incident,South of Market
29,PRIVATE,Medical Incident,Mission
...,...,...,...
1914,MEDIC,Medical Incident,Hayes Valley
1920,PRIVATE,Medical Incident,Tenderloin
1926,MEDIC,Medical Incident,South of Market
1935,MEDIC,Medical Incident,Oceanview/Merced/Ingleside


In [18]:
queries = ["What unit types are there?", "How many incidents are in Tenderloin?"]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', truncation=True, return_tensors="pt")

logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

IndexError: iloc cannot enlarge its target object

In [ ]:
encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

In [ ]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 
        'Age': ["56", "45", "59"],
        'Number of movies': ["87", "53", "69"],
        'Date of birth': ["7 february 1967", "10 june 1996", "28 november 1967"]}
queries = ["How many movies has George Clooney played in?", "How old is he?", "What's his date of birth?"]

table = pd.DataFrame.from_dict(data)

inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
logits = compute_prediction_sequence(model, inputs, device)

In [ ]:
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

queries = ["When was the most recent medical incident?", "How many incidents are in Tenderloin?"]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

#cfs_data_tapas=cfs_small[['call_number','call_date','call_type','unit_type','neighborhoods_analysis_boundaries']]
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

cfs_data_tapas

In [ ]:
queries = ["How many medical incidents?","What type of calls are there?" ]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

In [ ]:
logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering
#config = TapasConfig('google-base-finetuned-wikisql-supervised')
#model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base')

In [ ]:
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
config = TapasConfig.from_pretrained('google/tapas-base-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
# the base sized model with WikiSQL configuration
#config = TapasConfig('google-base-finetuned-wikisql-supervised')
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")


In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.sample(100)
#cfs_data_tapas=cfs_small[['call_type','neighborhoods_analysis_boundaries']]
#cfs_data_tapas=cfs_data_tapas.rename(columns={"call_type":"incident type","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_small

In [ ]:
list_of_list = [[]]
list_of_list[0] = list(cfs_small.columns)
list_of_list.extend(cfs_small.values.tolist())

In [ ]:
result = predict(list_of_list, ["How many medical incidents?","Which neighbour are there?"])

In [ ]:
queries = ["How many medical incidents?","Which neighbour are there?"]
inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

In [ ]:
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)